<a href="https://colab.research.google.com/github/FinOloughlin/CITS4403-Black-Death/blob/main/Copy_of_Black_Death.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

In [ ]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/AllenDowney/ThinkComplexity2/raw/master/notebooks/utils.py')
download('https://github.com/AllenDowney/ThinkComplexity2/raw/master/notebooks/Cell2D.py')

Downloaded utils.py
Downloaded Cell2D.py


In [ ]:
from utils import decorate, savefig
# make a directory for figures
!mkdir -p figs

In [ ]:
try:
    import empiricaldist
except ImportError:
    !pip install empiricaldist

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for empiricaldist: filename=empiricaldist-0.7.5-py3-none-any.whl size=12469 sha256=ef90bcc6afcec9d8240af48dd5bf873cf97961c3946f47170c49642b8d74df04
  Stored in directory: /root/.cache/pip/wheels/0d/d0/ae/1ad4c7593703e55b2321b23b49d3b0d55261b59d7036d7045b
Successfully built empiricaldist


### !! Mesa documentation: https://mesa.readthedocs.io/stable/

In [ ]:
!pip install mesa

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector


# Black Death

In [ ]:
class PlagueAgent(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.health_status = "Susceptible"
    self.age = self.random.randrange(1,80)
    self.days_infected = 0
    self.recovered = False
    self.dead = False

  def step(self):
    if self.health_status == "Infected":
      self.days_infected += 1

      if self.days_infected >= 4:
        self.check_recovery_or_death()

    self.move()

  def check_recovery_or_death(self):
    if self.age < 30:
      recovery_chance = 0.6
    elif 30 <= self.age <= 50:
      recovery_chance = 0.4
    else:
      recovery_chance = 0.2

    if self.age < 30:
      death_chance = 0.2
    elif 30 <= self.age <= 50:
      death_chance = 0.3
    else:
      death_chance = 0.5

    if not self.recovered and not self.dead:
      if self.random.random() < recovery_chance:
        self.recovered = True
        self.health_status = "Recovered"

    if not self.recovered and not self.dead:
      if self.random.random() < death_chance:
        self.dead = True
        self.health_status = "Dead"

  def move(self):
    possible_steps = self.model.grid.get_neighbourhood(
        self.pos,
        moore=True,
        include_center=True
    )

    new_position = self.random.choice(possible_steps)
    self.model.grid.move_agent(self, new_position)


class PlagueModel(Model):
  def __init__(self, width, height, num_agents):
        super().__init__()
        self.num_agents = num_agents
        self.height = height
        self.width = width
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        self.create_agents()

  def create_agents(self):
        for i in range(self.num_agents):
          agent = PlagueAgent(i, self)
          self.schedule.add(agent)

          x.self.random.randrange(self.grid.width)
          y.self.random.randrange(self.grid.height)
          self.grid.place_agent(agent, (x,y))

  def step(self):
    self.schedule.step()

  def plot_agents():
    pass